In [9]:
#1) supprimer les tirets et la reprtition de l'intitulé des variables

input_file = "LECLERC.csv"
output_file = "LECLERC_cleaned.csv"

with open(input_file, "r", encoding="utf-8") as f_in:
    lines = f_in.readlines()

cleaned_lines = []
header_encountered = False

for line in lines:
    # Suppression des lignes composées uniquement de tirets
    if line.strip().startswith('-'):
        continue

    # Suppression des répétitions d'en-tête (c-à-d. "Platform,Product Name,...") 
    # une fois qu'on l'a déjà rencontrée
    if line.startswith("Platform") and header_encountered:
        continue
    elif line.startswith("Platform"):
        header_encountered = True

    cleaned_lines.append(line)

with open(output_file, "w", encoding="utf-8") as f_out:
    f_out.writelines(cleaned_lines)

print("Fichier nettoyé et enregistré sous:", output_file)



Fichier nettoyé et enregistré sous: LECLERC_cleaned.csv


In [10]:
import pandas as pd

df = pd.read_csv('LECLERC_cleaned.csv')
df['Price'] = (df['Price']
               .str.replace(' €', '', regex=False)  # Supprime ' €'
               .str.replace(',', '.', regex=False)  # Remplace ',' par '.'
               .astype(float))                      # Convertit en float



In [11]:
from uuid import uuid4

#je rajoute un id pour chaque groupe product name-seller différent
def generate_uuid():
    return str(uuid4())

# Créer un dictionnaire de correspondance pour chaque combinaison unique
# On utilise groupby pour obtenir les groupes distincts et on attribue à chacun un UUID
group_mapping = {
    (row['Product Name'], row['Seller']): generate_uuid()
    for _, row in df[['Product Name', 'Seller']].drop_duplicates().iterrows()
}

# Ajouter une nouvelle colonne en mappant chaque ligne avec le UUID correspondant
df['ID'] = df.apply(lambda row: group_mapping[(row['Product Name'], row['Seller'])], axis=1)


In [12]:
# Écriture du DataFrame nettoyé dans un nouveau fichier CSV
df.to_csv(output_file, index=False)